In [1]:
import base64
import os
from PIL import Image
import glob
import json
import numpy as np

import PIL.Image

In [2]:
def calc_ratio (coords) :
    '''
    Args:
        coords: list of x-y coordinates of 4 coners [x1,y1,x2,y2,x3,y3,x4,y4]
    '''
    width = np.linalg.norm(np.array(coords[0:2])-np.array(coords[2:4]))
    height = np.linalg.norm(np.array(coords[2:4])-np.array(coords[4:6])) 
    
    return max(height, width) / min(height, width)

In [3]:
def save(
        filename,
        shapes,
        imagePath,
        imageHeight,
      
    imageWidth,
        imageData=None,
        otherData=None,
        flags=None,
    ):
    if otherData is None:
        otherData = {}
    if flags is None:
        flags = {}
    data = dict(
        version="4.5.9",
        flags=flags,
        shapes=shapes,
        imagePath=imagePath,
        imageData=imageData,
        imageHeight=imageHeight,
        imageWidth=imageWidth,
    )
    for key, value in otherData.items():
        assert key not in data
        data[key] = value
    try:
        with open(filename, "w") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        #self.filename = filename
    except Exception as e:
        raise LabelFileError(e)

In [7]:
ROOT = '/nas2/YJ/DATA/Trucks_max250_under3000//train/'
IMG_PATH = os.path.join(ROOT, 'images')
LBL_PATH = os.path.join(ROOT, 'labelTxt')

In [8]:
labels = glob.glob(os.path.join(LBL_PATH, '*.txt'))

In [ ]:
for label in labels :
    name = os.path.split(label)[1][:-4]
    IMG_NAME = '{}.png'.format(name)
    img = Image.open(os.path.join(IMG_PATH, IMG_NAME))
    w, h = img.size
    with open(label, 'r') as f :
        lines = f.readlines()
        shapes = []
        for line in lines :
            gt = line.split(' ')
            coords = [float(x) for x in gt[:8]]
            ratio = calc_ratio(coords)
            if ratio < 3 :
                cls = 'small_truck'
            else :
                cls = 'large_truck'
            data = {}
            data['label'] = cls
            data['points'] = [coords[0:2],coords[2:4],coords[4:6],coords[6:8]]
            data['group_id'] = None
            data['shape_type'] = "polygon"
            data["flasgs"] = {}
            shapes.append(data)
            
            save(
                filename=os.path.join(IMG_PATH, '{}.json'.format(name)),
                shapes=shapes,
                imagePath= name + '.png',
                imageHeight = h,
                imageWidth = w,
                imageData=None,
                otherData=None,
                flags=None,
                )